In [12]:
!pip install evaluate


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: D:\Python\Scripts\python.exe -m pip install --upgrade pip


In [10]:
!pip install sentence_transformers


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: D:\Python\Scripts\python.exe -m pip install --upgrade pip


In [8]:
!pip install peft


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: D:\Python\Scripts\python.exe -m pip install --upgrade pip


In [6]:
!pip install transformers

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   -------------------- ------------------- 5.2/10.4 MB 26.5 MB/s eta 0:00:01
   ---------------------------------------  10.2/10.4 MB 31.8 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 24.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 19.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: D:\Python\Scripts\python.exe -m pip install --upgrade pip


In [2]:
!pip install tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: D:\Python\Scripts\python.exe -m pip install --upgrade pip


In [4]:
!pip install datasets

   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ------- -------------------------------- 4.7/25.8 MB 22.0 MB/s eta 0:00:01
   --------------- ------------------------ 9.7/25.8 MB 23.2 MB/s eta 0:00:01
   ---------------------------- ----------- 18.1/25.8 MB 28.5 MB/s eta 0:00:01
   -------------------------------------- - 24.6/25.8 MB 28.9 MB/s eta 0:00:01
   ---------------------------------------- 25.8/25.8 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: D:\Python\Scripts\python.exe -m pip install --upgrade pip


In [3]:
!pip install google.colab

ERROR: Could not find a version that satisfies the requirement google.colab (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: D:\Python\Scripts\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for google.colab


In [2]:
import json
import pandas as pd
import numpy as np
import torch
import re
import logging
import sys
import os
from tqdm.auto import tqdm
import glob

# Import DataLoader (needed by Retriever class definition, even if not used in inference)
from torch.utils.data import DataLoader

# Import Dataset (needed by Generator class definition, even if not used in inference)
from datasets import Dataset

# Import Colab Drive for mounting
from google.colab import drive # Added import for Google Drive

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel
)
from sentence_transformers import SentenceTransformer, losses, models, util, InputExample, CrossEncoder
import evaluate

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(handler)


def load_corpus_from_squad_json(train_json_path, dev_json_path):
    logger.info(f"Loading corpus from local files: {train_json_path}, {dev_json_path}")

    def load_json_data(file_path):
        if not os.path.exists(file_path):
            logger.error(f"File not found: {file_path}")
            return None
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            if 'data' not in data:
                 logger.error(f"JSON file {file_path} does not contain the expected 'data' key.")
                 return None
            return data['data']
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding JSON from {file_path}: {e}")
            return None
        except Exception as e:
            logger.error(f"An unexpected error occurred loading {file_path}: {e}")
            return None

    train_data = load_json_data(train_json_path)
    dev_data = load_json_data(dev_json_path)

    if train_data is None and dev_data is None:
         logger.error("Failed to load both train and dev JSON files for corpus.")
         return pd.DataFrame()

    if train_data is None: train_data = []
    if dev_data is None: dev_data = []

    context_to_id = {}
    documents_list = []
    doc_id_counter = 0

    def process_paragraphs_for_corpus(data_split, desc):
        nonlocal doc_id_counter
        if not isinstance(data_split, list): return
        for article in tqdm(data_split, desc=f"Processing articles ({desc} corpus)"):
            if not isinstance(article, dict) or 'paragraphs' not in article or not isinstance(article['paragraphs'], list): continue
            for paragraph in article['paragraphs']:
                if not isinstance(paragraph, dict) or 'context' not in paragraph: continue
                context = paragraph['context']
                if not isinstance(context, str):
                     logger.warning(f"Context is not a string in {desc}, type: {type(context)}. Skipping.")
                     continue
                stripped_context = context.strip()
                if stripped_context not in context_to_id:
                    doc_id = f"doc_{doc_id_counter}"
                    context_to_id[stripped_context] = doc_id
                    documents_list.append({'id': doc_id, 'passage': context})
                    doc_id_counter += 1

    process_paragraphs_for_corpus(train_data, "train")
    process_paragraphs_for_corpus(dev_data, "dev")

    documents_df = pd.DataFrame(documents_list)
    logger.info(f"Loaded corpus with {len(documents_df)} unique passages.")

    if documents_df.empty:
         logger.error("Corpus is empty after processing.")
         return pd.DataFrame()

    return documents_df


class Retriever:
    def __init__(self, retriever_model_name="all-mpnet-base-v2", reranker_model_name="cross-encoder/ms-marco-MiniLM-L-6-v2"):
        logger.info(f"Initializing retriever model: {retriever_model_name}")
        self.retriever_model = SentenceTransformer(retriever_model_name)

        logger.info(f"Initializing reranker model: {reranker_model_name}")
        self.reranker_model = CrossEncoder(reranker_model_name)

        self.corpus = {}
        self.corpus_ids = []
        self.corpus_texts = []
        self.corpus_embeddings = None
        self.corpus_embeddings_device = None

        self.train_examples = []
        self.queries = {}
        self.relevant_docs = {}

    def load_corpus(self, documents_df):
        logger.info("Loading corpus documents...")
        documents_df['id'] = documents_df['id'].astype(str)
        documents_df['passage'] = documents_df['passage'].fillna("").astype(str)

        self.corpus = dict(zip(documents_df['id'], documents_df['passage']))
        self.corpus_ids = list(self.corpus.keys())
        self.corpus_texts = list(self.corpus.values())

        logger.info(f"Corpus size: {len(self.corpus)} documents.")

    def precompute_corpus_embeddings(self, batch_size=1024):
        if not self.corpus_texts:
            raise ValueError("No corpus found. Did you run load_corpus(...) first.")

        logger.info(f"Computing embeddings for {len(self.corpus_texts)} passages...")
        all_embeddings = []

        device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"Encoding device: {device}")
        self.retriever_model.to(device)

        try:
            for start_idx in tqdm(range(0, len(self.corpus_texts), batch_size), desc="Encoding corpus"):
                batch = self.corpus_texts[start_idx:start_idx + batch_size]
                batch = [str(t) for t in batch]
                batch_embeddings = self.retriever_model.encode(
                    batch,
                    convert_to_tensor=True,
                    show_progress_bar=False,
                    device=device
                )
                all_embeddings.append(batch_embeddings)

            self.corpus_embeddings = torch.cat(all_embeddings, dim=0)
            self.corpus_embeddings_device = self.corpus_embeddings.device
            logger.info(f"Corpus embeddings shape: {self.corpus_embeddings.shape} on device: {self.corpus_embeddings_device}")

        except Exception as e:
            logger.error(f"Error during corpus encoding: {e}")
            self.corpus_embeddings = None
            self.corpus_embeddings_device = None
            raise

    def re_rank_passages(self, query, initial_results):
        if not initial_results: return []
        cross_encoder_input = [[str(query), str(res['passage'])] for res in initial_results]
        if not cross_encoder_input: return []

        try:
            rerank_scores = self.reranker_model.predict(cross_encoder_input)
            for i, res in enumerate(initial_results):
                res['rerank_score'] = float(rerank_scores[i])
            reranked_results = sorted(initial_results, key=lambda x: x.get('rerank_score', -float('inf')), reverse=True)
            return reranked_results
        except Exception as e:
            logger.error(f"Error during re-ranking: {e}")
            logger.warning("Re-ranking failed. Returning initial results sorted by original score.")
            return sorted(initial_results, key=lambda x: x.get('score', -float('inf')), reverse=True)


    def retrieve_top_k(self, query, top_k=5, initial_retrieval_k=100, use_reranking=True):
        if self.corpus_embeddings is None or self.corpus_embeddings_device is None:
            logger.warning("Corpus embeddings not precomputed or device unknown. Attempting precomputation...")
            try:
                self.precompute_corpus_embeddings()
            except Exception as e:
                 logger.error(f"Failed to compute corpus embeddings: {e}. Cannot perform retrieval.")
                 return []
            if self.corpus_embeddings is None or self.corpus_embeddings_device is None:
                 logger.error("Corpus embeddings still not available after attempted precomputation. Cannot perform retrieval.")
                 return []


        device = self.corpus_embeddings_device
        self.retriever_model.to(device)
        query_embedding = self.retriever_model.encode(str(query), convert_to_tensor=True, device=device)

        cos_scores = util.cos_sim(query_embedding, self.corpus_embeddings)[0]

        initial_k = initial_retrieval_k if use_reranking else top_k
        initial_k = min(initial_k, len(self.corpus_texts))
        initial_k = max(1, initial_k) if len(self.corpus_texts) > 0 else 0

        if initial_k == 0:
             logger.warning("Corpus is empty or initial_k is 0. Cannot retrieve.")
             return []

        top_values, top_indices = torch.topk(cos_scores, k=initial_k)
        top_indices_list = top_indices.cpu().numpy()

        initial_retrieved_passages = []
        corpus_ids_str = [str(cid) for cid in self.corpus_ids]

        for idx in top_indices_list:
            cid = corpus_ids_str[idx]
            passage = self.corpus.get(cid, "")
            initial_retrieved_passages.append({
                'corpus_id': cid,
                'passage': passage,
                'score': cos_scores[idx].item()
            })

        if use_reranking:
            reranked_results = self.re_rank_passages(query, initial_retrieved_passages)
            return reranked_results[:top_k]
        else:
            return sorted(initial_retrieved_passages, key=lambda x: x.get('score', -float('inf')), reverse=True)[:top_k]


class Generator:
    pass

class RAGSystem:
    def __init__(self, retriever_model_path=None, generator_model_path=None,
                 retriever_base_model="all-mpnet-base-v2", reranker_base_model="cross-encoder/ms-marco-MiniLM-L-6-v2",
                 generator_base_model="gpt2"):
        logger.info("Initializing RAG System...")

        if retriever_model_path and os.path.exists(retriever_model_path):
             logger.info(f"Loading trained retriever model from {retriever_model_path}")
             try:
                 self.retriever = Retriever(retriever_model_name=retriever_model_path, reranker_model_name=reranker_base_model)
                 logger.info("Trained Retriever model loaded.")
             except Exception as e:
                 logger.error(f"Failed to load trained retriever model from {retriever_model_path}: {e}")
                 logger.warning(f"Proceeding by loading base retriever model {retriever_base_model}.")
                 self.retriever = Retriever(retriever_base_model, reranker_base_model)
        else:
             logger.warning(f"Retriever model path not provided or does not exist: {retriever_model_path}")
             logger.info(f"Loading base retriever model {retriever_base_model}.")
             self.retriever = Retriever(retriever_base_model, reranker_base_model)


        logger.info(f"Loading generator model from {generator_model_path}...")

        latest_checkpoint = None
        if generator_model_path and os.path.exists(generator_model_path):
            checkpoint_dirs = glob.glob(os.path.join(generator_model_path, 'checkpoint-*'))
            if checkpoint_dirs:
                try:
                    latest_checkpoint = max(checkpoint_dirs, key=lambda x: int(os.path.basename(x).split('-')[1]))
                    logger.info(f"Found latest checkpoint: {latest_checkpoint}")
                except Exception as e:
                    logger.warning(f"Could not determine latest checkpoint from names: {e}. Using the main generator_model_path directly if it contains model files.")
                    latest_checkpoint = generator_model_path


        model_load_path = latest_checkpoint if latest_checkpoint and os.path.exists(latest_checkpoint) else generator_base_model

        try:
            self.generator_model = AutoModelForCausalLM.from_pretrained(model_load_path)
            self.generator_tokenizer = AutoTokenizer.from_pretrained(model_load_path)
            logger.info(f"Generator model loaded from {model_load_path}.")

            if self.generator_tokenizer.pad_token is None:
                 self.generator_tokenizer.pad_token = self.generator_tokenizer.eos_token
            self.generator_model.config.pad_token_id = self.generator_tokenizer.eos_token_id

        except Exception as e:
            logger.error(f"Failed to load generator model from {model_load_path}: {e}")
            logger.warning(f"Proceeding by loading base generator model {generator_base_model}.")
            try:
                self.generator_model = AutoModelForCausalLM.from_pretrained(generator_base_model)
                self.generator_tokenizer = AutoTokenizer.from_pretrained(generator_base_model)
                if self.generator_tokenizer.pad_token is None:
                     self.generator_tokenizer.pad_token = self.generator_tokenizer.eos_token
                self.generator_model.config.pad_token_id = self.generator_tokenizer.eos_token_id
                logger.info(f"Base generator model {generator_base_model} loaded instead.")
            except Exception as base_e:
                logger.error(f"Failed to load base generator model {generator_base_model} either: {base_e}")
                self.generator_model = None
                self.generator_tokenizer = None
                logger.error("Generator model could not be loaded. Generation will not work.")


        self.corpus_doc_dict = {}


    def load_corpus(self, documents_df):
        logger.info("Loading corpus into RAG system...")
        if documents_df.empty:
             logger.warning("Corpus DataFrame is empty.")
             self.corpus_doc_dict = {}
             self.retriever.corpus = {}
             self.retriever.corpus_ids = []
             self.retriever.corpus_texts = []
        else:
            documents_df["id"] = documents_df["id"].astype(str)
            documents_df["passage"] = documents_df["passage"].fillna("").astype(str)
            self.corpus_doc_dict = dict(zip(documents_df["id"], documents_df["passage"]))
            self.retriever.load_corpus(documents_df)
            logger.info("Corpus loaded into RAG system's internal storage.")


    def precompute_retriever_embeddings(self, batch_size=1024):
        if not self.retriever.corpus_texts:
             logger.warning("Retriever corpus is empty. Cannot precompute embeddings.")
             return

        logger.info("Starting precomputation of retriever embeddings...")
        try:
            self.retriever.precompute_corpus_embeddings(batch_size=batch_size)
            logger.info("Retriever embeddings precomputation complete.")
        except Exception as e:
            logger.error(f"Retriever embeddings precomputation failed: {e}")


    def answer_question(self, question: str, top_k_retrieval=5, initial_retrieval_k=100, use_reranking=True,
                        max_new_tokens=100, do_sample=False):
        if self.retriever.corpus_embeddings is None or self.retriever.corpus_embeddings_device is None:
            logger.error("Retriever corpus embeddings not available. Cannot perform retrieval.")
            return "Error: Retriever embeddings not loaded or failed to compute."

        if not self.corpus_doc_dict:
             logger.error("Corpus document dictionary not loaded. Cannot retrieve passage texts.")
             return "Error: Corpus not loaded."

        if self.generator_model is None or self.generator_tokenizer is None:
             logger.error("Generator model or tokenizer not loaded. Cannot perform generation.")
             return "Error: Generator model not loaded."


        logger.info(f"Processing question: '{question}'")

        logger.info(f"Retrieving top {top_k_retrieval} passages...")
        retrieved_results = self.retriever.retrieve_top_k(
            question,
            top_k=top_k_retrieval,
            initial_retrieval_k=initial_retrieval_k,
            use_reranking=use_reranking
        )

        if not retrieved_results:
            logger.warning("No passages retrieved.")
            combined_context = ""
            logger.info("Generating with empty context.")
        else:
            combined_context = " ".join([self.corpus_doc_dict.get(res.get('corpus_id', ''), '') for res in retrieved_results])

            if not combined_context.strip():
                 logger.warning("Retrieved passages are empty or contain no text after lookup.")
                 combined_context = ""
                 logger.info("Generating with empty context.")
            else:
                 logger.info(f"Combined context length: {len(combined_context)} characters.")


        logger.info("Generating answer...")

        model = self.generator_model
        tokenizer = self.generator_tokenizer
        max_positions = model.config.max_position_embeddings

        max_new_tokens_gen = max_new_tokens
        allowed_prompt_length = max_positions - max_new_tokens_gen
        allowed_prompt_length = max(1, allowed_prompt_length)


        q_part_template = f"Question: {question}\nContext: "
        a_part_template = "\nAnswer: "

        q_tokens_len = len(tokenizer.encode(q_part_template, add_special_tokens=False))
        a_tokens_len = len(tokenizer.encode(a_part_template, add_special_tokens=False))

        fixed_length = q_tokens_len + a_tokens_len
        allowed_for_context = max(0, allowed_prompt_length - fixed_length)

        context_to_truncate = combined_context

        if not isinstance(context_to_truncate, str) or not context_to_truncate.strip():
             truncated_context = ""
        else:
             context_tokens = tokenizer.encode(context_to_truncate, add_special_tokens=False)
             truncated_context_tokens = context_tokens[:allowed_for_context]
             truncated_context = tokenizer.decode(truncated_context_tokens, skip_special_tokens=True)

        prompt = f"Question: {question}\nContext: {truncated_context}\nAnswer: "

        final_prompt_length = len(tokenizer.encode(prompt, add_special_tokens=True))
        if final_prompt_length > allowed_prompt_length:
             logger.warning(f"Final prompt length ({final_prompt_length}) exceeds allowed ({allowed_prompt_length}) before generation.")
             prompt_tokens = tokenizer.encode(prompt, add_special_tokens=True)
             truncated_prompt_tokens = prompt_tokens[:allowed_prompt_length]
             prompt = tokenizer.decode(truncated_prompt_tokens, skip_special_tokens=True)
             logger.warning(f"Further truncated prompt to {len(tokenizer.encode(prompt, add_special_tokens=True))} tokens.")


        try:
            model.eval()
            device = "cuda" if torch.cuda.is_available() else "cpu"
            model.to(device)


            gen_pipe = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                device=0 if device == "cuda" else -1,
            )

            generation_params = {
                "max_new_tokens": max_new_tokens_gen,
                "num_return_sequences": 1,
                "pad_token_id": tokenizer.eos_token_id,
                "eos_token_id": tokenizer.eos_token_id,
                "return_full_text": False
            }
            if do_sample:
                generation_params["do_sample"] = True
                generation_params["temperature"] = 0.7
                generation_params["top_k"] = 50
                generation_params["top_p"] = 0.95
            else:
                 generation_params["do_sample"] = False

            gen_output = gen_pipe(
                prompt,
                **generation_params
            )
            logger.debug(f"Raw pipeline output: {gen_output}")


            if gen_output and isinstance(gen_output, list) and len(gen_output) > 0 and 'generated_text' in gen_output[0]:
                 generated_text = gen_output[0]['generated_text'].strip()
                 if "Answer: " in generated_text:
                      generated_text = generated_text.split("Answer: ")[-1].strip()
                 elif "Answer:" in generated_text:
                      generated_text = generated_text.split("Answer:")[-1].strip()


            else:
                 generated_text = ""
                 logger.warning(f"Pipeline returned unexpected output for prompt starting '{prompt[:50]}...'. Output: {gen_output}")

            if torch.cuda.is_available():
                 torch.cuda.empty_cache()


            final_answer = generated_text

            return final_answer

        except Exception as e:
            logger.error(f"Error during generation: {e}")
            if torch.cuda.is_available(): torch.cuda.empty_cache()
            return "Error: Could not generate answer."


if __name__ == "__main__":
    # --- Colab Specific Setup ---
    # Mount Google Drive to access data and models
    try:
        drive.mount('/content/drive')
        logger.info("Google Drive mounted successfully.")
        # Define base path for data and models within your Drive
        # *** UPDATE THIS PATH TO WHERE YOUR train_data.json, dev_data.json, and output folder are stored in Drive ***
        # Example: If your files are in 'My Drive/Colab_RAG_Project/'
        drive_base_path = '/content/drive/MyDrive/Colab_RAG_Project' # <-- UPDATE THIS PATH
        train_json_file = os.path.join(drive_base_path, 'train_data.json')
        dev_json_file = os.path.join(drive_base_path, 'dev_data.json')
        retriever_model_path = os.path.join(drive_base_path, "output/retriever-model")
        generator_model_path = os.path.join(drive_base_path, "output/generator-finetuned")

        # Ensure output directories exist (within Drive)
        os.makedirs(retriever_model_path, exist_ok=True)
        os.makedirs(generator_model_path, exist_ok=True)


    except Exception as e:
        logger.error(f"Failed to mount Google Drive or set paths: {e}")
        logger.warning("Proceeding with local paths. Ensure data and models are in the local Colab environment.")
        # Fallback to local paths if Drive mounting fails
        train_json_file = 'train_data.json'
        dev_json_file = 'dev_data.json'
        retriever_model_path = "output/retriever-model"
        generator_model_path = "output/generator-finetuned"

        # Ensure local output directories exist
        os.makedirs(retriever_model_path, exist_ok=True)
        os.makedirs(generator_model_path, exist_ok=True)


    # --- Check for GPU Availability ---
    if torch.cuda.is_available():
        logger.info(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
        # Note: Selecting the specific GPU type (like A100) is done in Colab's Runtime settings:
        # Runtime -> Change runtime type -> Hardware accelerator -> GPU -> Choose GPU type (e.g., A100)
    else:
        logger.warning("CUDA is not available. Using CPU. This will be significantly slower.")


    # Define base models (needed for loading if trained models not found/fail to load)
    retriever_base_model = "all-mpnet-base-v2"
    reranker_base_model = "cross-encoder/ms-marco-MiniLM-L-6-v2"
    generator_base_model = "gpt2" # Base GPT-2 model name


    # --- Load Corpus ---
    logger.info("--- Loading Corpus Data for Inference ---")
    corpus_documents_df = load_corpus_from_squad_json(train_json_file, dev_json_file)

    if corpus_documents_df.empty:
        logger.error("Corpus loading failed or resulted in empty documents_df. Cannot run RAG system. Exiting.")
        sys.exit(1)

    logger.info("--- Corpus Loading Complete ---")

    # --- Initialize and Setup RAG System ---
    logger.info("--- Initializing RAG System ---")
    try:
        rag_system = RAGSystem(
            retriever_model_path=retriever_model_path,
            generator_model_path=generator_model_path, # Pass the directory containing checkpoint folders
            retriever_base_model=retriever_base_model,
            reranker_base_model=reranker_base_model,
            generator_base_model=generator_base_model # Pass the base model name
        )

        rag_system.load_corpus(corpus_documents_df)

        logger.info("Precomputing retriever embeddings for RAG inference...")
        # Adjust batch size based on your GPU memory (A100 can handle larger batches)
        rag_system.precompute_retriever_embeddings(batch_size=2048) # Increased batch size example
        if rag_system.retriever.corpus_embeddings is None:
            logger.error("Retriever embeddings precomputation failed. Cannot run RAG system. Exiting.")
            sys.exit(1)

    except Exception as e:
        logger.error(f"Failed to initialize or setup RAG System: {e}. Exiting.")
        sys.exit(1)


    logger.info("--- RAG System Ready ---")
    print("\nType your question and press Enter.")
    print("Type 'quit' or 'exit' to leave the chat.")
    print("-" * 50)

    # --- Command-Line Interface Loop ---
    while True:
        try:
            user_question = input("Enter your question: ")

            if user_question.lower() in ['quit', 'exit']:
                print("Exiting chat. Goodbye!")
                break

            if not user_question.strip():
                 print("Please enter a question.")
                 continue

            answer = rag_system.answer_question(
                user_question,
                top_k_retrieval=5, # Number of passages to retrieve
                initial_retrieval_k=50, # Candidates before reranking
                use_reranking=True, # Use reranking
                max_new_tokens=150, # Max length of the generated answer
                do_sample=False # Set to True for less deterministic answers
            )

            print(f"\nAnswer: {answer}\n")
            print("-" * 50)

        except EOFError:
            print("\nExiting chat. Goodbye!")
            break
        except Exception as e:
            logger.error(f"An error occurred during answering: {e}")
            print("An error occurred while processing your question. Please try again.")
            print("-" * 50)


    # Optional: Clean up models from memory/GPU when exiting
    if torch.cuda.is_available():
        if hasattr(rag_system, 'retriever') and hasattr(rag_system.retriever, 'retriever_model') and rag_system.retriever.retriever_model is not None:
             try: rag_system.retriever.retriever_model.cpu()
             except: pass
             if hasattr(rag_system.retriever, 'reranker_model'):
                 try: rag_system.retriever.reranker_model.model.to('cpu')
                 except: pass
        if hasattr(rag_system, 'generator_model') and rag_system.generator_model is not None:
            try: rag_system.generator_model.to('cpu')
            except: pass
        torch.cuda.empty_cache()



ModuleNotFoundError: No module named 'google.colab'